# Lecture des fichiers et Machine Learning

# 1. import des bibliothèques et lecture des fichiers

In [8]:
import time
import pandas as pd
import numpy as np
import glob

from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

In [9]:
# file_list= glob.glob('variables/data_kmeans1_sample200-*.pkl')
# file_list

In [10]:
# start = time.time()

# frames = []
# for pkl_file in file_list:
#     new_df=pd.read_pickle(pkl_file)
#     frames.append(new_df)
# data = pd.concat(frames,axis=0)

# end = time.time()
# elapsed = end - start
# print(f'Time elapsed : {elapsed//60}min {round(elapsed % 60,0)}sec for {len(data):0} images (= {round(len(data)/17092*100,2):0}% of the total dataset).')

In [11]:
# pour ignorer le problème de mémoire souvent rencontré
import warnings
warnings.filterwarnings("ignore")

start = time.time()

# on lit le csv et le transpose directement
data=pd.read_csv('variables/test_save_kmeans_sample200.csv',index_col=0).T

end = time.time()
elapsed = end - start
print(f'Time elapsed : {elapsed//60}min {round(elapsed % 60,0)}sec for {len(data):0} images (= {round(len(data)/17092*100,2):0}% of the total dataset).')

Time elapsed : 1.0min 30.0sec for 1600 images (= 9.36% of the total dataset).


In [12]:
data.head()

,img_name,process_time,label,p1,p2,p3,p4,p5,p6,p7,...,p130671,p130672,p130673,p130674,p130675,p130676,p130677,p130678,p130679,p130680
0,SNE_77438.jpg,1.434381,NE - neutrophils,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0
1,BNE_260247.jpg,1.084768,NE - neutrophils,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0
2,SNE_414238.jpg,0.959654,NE - neutrophils,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0
3,SNE_756802.jpg,0.967628,NE - neutrophils,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0
4,BNE_686296.jpg,0.955025,NE - neutrophils,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0


In [13]:
data['label'].value_counts()

NE - neutrophils              200
EO - eosinophils              200
IG - immature granulocytes    200
PL - thrombocytes             200
ER - erythroblasts            200
MO - monocytes                200
BA - basophils                200
LY - lymphocytes              200
Name: label, dtype: int64

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1600 entries, 0 to 1599
Columns: 130683 entries, img_name to p130680
dtypes: object(130683)
memory usage: 1.6+ GB


## 2. Machine Learning

### Train test split

In [15]:
target = data['label'].apply(lambda x: x.split(' - ')[0])
feats = data.iloc[:,3:]

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(feats, target, test_size=0.2,shuffle=True,random_state=123)

### Support Vector Machine

In [17]:
start = time.time()

# model=svm.SVC(kernel='poly'  #{'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'}, default='rbf'
#             ,gamma=0.01  #{'scale', 'auto'} or float, default='scale'
#            )
model=svm.SVC()

model.fit(X_train, y_train)
y_pred=model.predict(X_test)
cm = pd.crosstab (y_test, y_pred, rownames=['Reality'], colnames=['Prediction'])
display(cm)
accuracy=model.score(X_test, y_test) # retourne l'accuracy directement
# print('Accuracy :', accuracy)

print(classification_report(y_test, y_pred))

end = time.time()
elapsed = end - start
print(f'\nTime elapsed : {elapsed//60 :0}min {round(elapsed % 60,0):0}sec')

Prediction,BA,EO,ER,IG,LY,MO,NE,PL
Reality,,,,,,,,
BA,28,1,0,4,1,1,0,0
EO,1,25,1,1,0,3,5,0
ER,1,1,28,2,1,0,0,1
IG,4,1,1,20,5,2,2,0
LY,4,0,1,1,37,0,2,0
MO,8,4,0,19,0,22,0,0
NE,0,3,2,0,0,0,31,0
PL,0,0,1,0,0,0,0,45


              precision    recall  f1-score   support

          BA       0.61      0.80      0.69        35
          EO       0.71      0.69      0.70        36
          ER       0.82      0.82      0.82        34
          IG       0.43      0.57      0.49        35
          LY       0.84      0.82      0.83        45
          MO       0.79      0.42      0.54        53
          NE       0.78      0.86      0.82        36
          PL       0.98      0.98      0.98        46

    accuracy                           0.74       320
   macro avg       0.74      0.75      0.73       320
weighted avg       0.76      0.74      0.73       320


Time elapsed : 12.0min 4.0sec


### Random Forest

In [18]:
start = time.time()

model = RandomForestClassifier()

model.fit(X_train, y_train)
y_pred=model.predict(X_test)
cm = pd.crosstab (y_test, y_pred, rownames=['Reality'], colnames=['Prediction'])
display(cm)
accuracy=model.score(X_test, y_test) # retourne l'accuracy directement

# print('Accuracy :', accuracy)

print(classification_report(y_test, y_pred))

end = time.time()
elapsed = end - start
print(f'Time elapsed : {elapsed//60 :0}min {round(elapsed % 60,0):0}sec')

Prediction,BA,EO,ER,IG,LY,MO,NE,PL
Reality,,,,,,,,
BA,31,1,0,2,1,0,0,0
EO,2,23,0,3,0,5,2,1
ER,2,0,29,0,1,0,2,0
IG,2,2,0,21,4,4,2,0
LY,4,0,3,4,32,2,0,0
MO,3,2,0,8,0,39,1,0
NE,0,1,2,1,0,0,32,0
PL,0,0,0,0,0,0,0,46


              precision    recall  f1-score   support

          BA       0.70      0.89      0.78        35
          EO       0.79      0.64      0.71        36
          ER       0.85      0.85      0.85        34
          IG       0.54      0.60      0.57        35
          LY       0.84      0.71      0.77        45
          MO       0.78      0.74      0.76        53
          NE       0.82      0.89      0.85        36
          PL       0.98      1.00      0.99        46

    accuracy                           0.79       320
   macro avg       0.79      0.79      0.79       320
weighted avg       0.80      0.79      0.79       320

Time elapsed : 0.0min 28.0sec
